# OpenVINO使ってMobileNetモデルを推論するデモ

## まずはオリジナルのモデル（Keras + Tensorflow）の推論用関数を定義

In [ ]:
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras import models
import numpy as np
import glob
import random
import time
import pandas as pd

def inference_original(total = 100):
    model = models.load_model('top_layers.mn.hdf5')
    
    #Read in Labels
    arg_labels="mn-labels.txt"
    label_file = open(arg_labels, "r")
    labels = label_file.read().split('\n')
    
    list_df = pd.DataFrame( columns=['正解ラベル','予測ラベル','全処理時間(msec)','推論時間(msec)'] )

    total_spent_time = 0
    total_infer_spent_time = 0
    
    print(str(total) + '枚の画像を処理中．．．')
    for i in range(total):
        time1 = time.time()
        file_list = glob.glob("test/*/*")
        img_path = random.choice(file_list)
        img_cat = os.path.split(os.path.dirname(img_path))[1]
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        time2 = time.time()
        preds = model.predict(x)
        
        infer_spent_time = time.time() - time2
        total_infer_spent_time += infer_spent_time
        
        spent_time = time.time() - time1
        total_spent_time += spent_time
        
        top = preds[0].argsort()[-1:][::-1]
        pred_label = labels[top[0]]
        print("*", end="")
        #print(str(i + 1) + '枚目: 正解=' + img_cat + '、予測=' + pred_label + '、全処理時間=' + str(int(spent_time * 1000)) + 'msec、推論時間=' + str(int(infer_spent_time * 1000)) + ' msec')
        #print("Raw Predictions: ", preds[0])
        tmp_se = pd.Series( [img_cat, pred_label, str(int(spent_time * 1000)), str(int(infer_spent_time * 1000)) ], index=list_df.columns )
        list_df = list_df.append( tmp_se, ignore_index=True )        

    print()
    print('完了！')
    print()
    print("平均処理時間: " + str(int((total_spent_time / total)*1000.0)) + " ms/枚")
    print("平均推論時間: " + str(int((total_infer_spent_time / total)*1000.0)) + " ms/枚")
    display(list_df)

## 続いて、オリジナルのモデルをOpenVINOのIR形式に変換

OpenVINOに付属しているModel Optimizerを用います。
大前提として、OSの環境変数に下記がセットされている必要があります。

PYTHONPATH=c:\Program Files (x86)\IntelSWTools\openvino\python\python3.6;c:\Program Files (x86)\IntelSWTools\openvino\python\python3;

より詳しい説明は下記URLをご覧ください。 https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_convert_model_Converting_Model.html

In [ ]:
!python "c:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo.py" --input_model=tf_model\top_layers.mn.pb --input_shape=[1,224,224,3]

モデルが正常に変換されると、xmlファイルとbinファイルが出来上がります。これがOpenVINOで用いられるIR形式のモデルの実体です。

## ここで、IR形式のモデルを推論処理するための関数を定義

In [ ]:
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras import models
import numpy as np
import glob
import random
import time
import os
import cv2
import pandas as pd

from openvino.inference_engine import IENetwork, IEPlugin

def inference_openvino(total = 100, target_device="CPU"):
    model_xml = 'top_layers.mn.xml'
    model_bin = 'top_layers.mn.bin'

    # Plugin initialization for specified device and load extensions library if specified
    ie = IEPlugin(device=target_device, plugin_dirs='')

    # Read IR
    net = IENetwork(model=model_xml, weights=model_bin)

    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
    net.batch_size = 1

    # Loading model to the plugin
    exec_net = ie.load(network=net)
    
    #Read in Labels
    arg_labels="mn-labels.txt"
    label_file = open(arg_labels, "r")
    labels = label_file.read().split('\n')
    
    list_df = pd.DataFrame( columns=['正解ラベル','予測ラベル','全処理時間(msec)','推論時間(msec)'] )

    total_spent_time = 0
    total_infer_spent_time = 0
    
    print(str(total) + '枚の画像を処理中．．．')
    for j in range(total):
        time1 = time.time()
        file_list = glob.glob("test/*/*")
        img_path = random.choice(file_list)
        img_cat = os.path.split(os.path.dirname(img_path))[1]
        # Read and pre-process input images
        n, c, h, w = net.inputs[input_blob].shape
        images = np.ndarray(shape=(n, c, h, w))
        for i in range(n):
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if image.shape[:-1] != (h, w):
                image = cv2.resize(image, (w, h))
            image = image.transpose((2, 0, 1))  # Change data layout from HWC to CHW
            image = image.reshape((n, c, h, w))
            image = preprocess_input(image)
            images[i] = image

        # Start sync inference
        time2 = time.time()
        preds = exec_net.infer(inputs={input_blob: images})
        
        infer_spent_time = time.time() - time2
        total_infer_spent_time += infer_spent_time
        
        spent_time = time.time() - time1
        total_spent_time += spent_time
        
        preds = preds[out_blob]
        top = preds[0].argsort()[-1:][::-1]
        pred_label = labels[top[0]]
        print("*", end="")
        #print(str(j + 1) + '枚目: 正解=' + img_cat + '、予測=' + pred_label + '、全処理時間=' + str(int(spent_time * 1000)) + 'msec、推論時間=' + str(int(infer_spent_time * 1000)) + ' msec')
        #print("Raw Predictions: ", preds)
        tmp_se = pd.Series( [img_cat, pred_label, str(int(spent_time * 1000)), str(int(infer_spent_time * 1000)) ], index=list_df.columns )
        list_df = list_df.append( tmp_se, ignore_index=True ) 

    print()
    print('完了！')
    print()
    print("平均処理時間: " + str(int((total_spent_time / total)*1000.0)) + " ms/枚")
    print("平均推論時間: " + str(int((total_infer_spent_time / total)*1000.0)) + " ms/枚")
    display(list_df)

## では、2つの関数を実行して性能を比較しましょう

## まずは、オリジナルのモデルから実行

In [ ]:
inference_original(total=50)

## 続いて、OpenVINOのIR形式のモデルを実行

In [ ]:
inference_openvino(total=50, target_device="CPU")

## 同じIR形式のモデルを内臓GPUに推論処理をオフロードして実行

In [ ]:
inference_openvino(total=50, target_device="GPU")

## いかがでしたでしょうか？
## これより下は、この2つのモデルを用いた動画データに対する推論性能の比較です。

# OpenVINO Video Inference on CPU

Copyright (c) 2019 Intel Corporation.

Permission is hereby granted, free of charge, to any person obtaining
a copy of this software and associated documentation files (the
"Software"), to deal in the Software without restriction, including
without limitation the rights to use, copy, modify, merge, publish,
distribute, sublicense, and/or sell copies of the Software, and to
permit persons to whom the Software is furnished to do so, subject to
the following conditions:

The above copyright notice and this permission notice shall be
included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE
LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION
WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


# Objective 
Transform your frozen graph into an intermediate representation (.bin/.xml) needed to use with OpenVINO then instantiate your OpenVINO model and inference live on a video file.

# Activities 
**In this section of the training you will**
- Understand OpenVINO Arguments
- Instantiate OpenVINO Network
- Use OpenCV to read video and pass frames to OpenVINO

Click the cell below and then click **Run**.

In [ ]:
import cv2
import numpy as np
from PIL import Image
import PIL
import io
import IPython.display
from IPython.display import clear_output

### Instantiate Network

Next, we'll instantiate our network.  If you want to take a closer look at all the specific steps required to instantitate the network look at the inference.py file included in this directory.  Instead, we're going to look at the parameters passed to the constructor.  We see the .XML file passed as the base model, no CPU extensions and targeting the CPU as the device type.

Then we'll call out to the constructor for the Network instantiation. We then want to load our model into that network by passing in the above parameters to load_model. Lastly, we'll read in our labels file that we're going to use to decode the results during our inference.

Click the cell below and then click **Run**.

In [ ]:
from inference import Network
import sys

arg_model="top_layers.mn.xml"
arg_cpu_extension=None
arg_device="GPU"

# Initialise the class
infer_network = Network()
# Load the network to IE plugin to get shape of input layer
plugin, (n_fd, c_fd, h_fd, w_fd) = infer_network.load_model(arg_model, arg_device, 1, 1, 0, arg_cpu_extension)

#Read in Labels
arg_labels="mn-labels.txt"
label_file = open(arg_labels, "r")
labels = label_file.read().split('\n')

Now for the inference!  There are 8 steps that we're doing in the code below:
- Start a video or webcam capture
- Read the video frame
- Put the classification text on to the frame
- Render the frame to the Cell output field
- Resize/Transpose/Reshape/Preprocess frame
- Start an inference request
- Interpret the inference result
- Clear frame from notebook cell

Click the cell below and then click **Run**.

In [ ]:
from keras.applications.mobilenet import preprocess_input
import time

cap = cv2.VideoCapture("persian.mp4")
pred_label = ""
fps = 0
ips = 0
while True:
    time1 = time.time()
    ret, next_frame = cap.read()
    
    if ret:
        next_frame = cv2.cvtColor(next_frame, cv2.COLOR_BGR2RGB)

        cv2.putText(next_frame,str(pred_label) + " " + str(fps) + "fps " + str(ips) + "ips", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,2550), 4)
        cv2.putText(next_frame,str(pred_label) + " " + str(fps) + "fps " + str(ips) + "ips", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)

        f = io.BytesIO()
        PIL.Image.fromarray(next_frame).save(f, 'jpeg')
        IPython.display.display(IPython.display.Image(data=f.getvalue()))

        in_frame_fd = cv2.resize(next_frame, (w_fd, h_fd))
        in_frame_fd = in_frame_fd.transpose((2, 0, 1))
        in_frame_fd = in_frame_fd.reshape((n_fd, c_fd, h_fd, w_fd))
        in_frame_fd = preprocess_input(in_frame_fd)

        time3 = time.time()
        # Start asynchronous inference for specified request
        infer_network.exec_net(0, in_frame_fd)
        # Wait for the result
        infer_network.wait(0)
        # Results of the output layer of the network
        res = infer_network.get_output(0)
        time4 = time.time()

        top = res[0].argsort()[-1:][::-1]
        pred_label = labels[top[0]]

        time2 = time.time()
        fps = '%.1f' % (1/(time2-time1))
        ips = '%.1f' % (1/(time4-time3))

        clear_output(wait=True)
    else:
        break

clear_output(wait=True)
print("Video Ended")
infer_network.clean()
cap.release()
cv2.destroyAllWindows()

Below is the case that Original model is used for the same video data. You probably can see the difference of performance.

In [ ]:
from keras.applications.mobilenet import preprocess_input
import time
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras import models
import numpy as np
import glob
import random

model = models.load_model('top_layers.mn.hdf5')

cap = cv2.VideoCapture("persian.mp4")
pred_label = ""
fps = 0
ips = 0
while True:
    time1 = time.time()
    ret, next_frame = cap.read()
    
    if ret:
        next_frame = cv2.cvtColor(next_frame, cv2.COLOR_BGR2RGB)

        cv2.putText(next_frame,str(pred_label) + " " + str(fps) + "fps " + str(ips) + "ips", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,2550), 4)
        cv2.putText(next_frame,str(pred_label) + " " + str(fps) + "fps " + str(ips) + "ips", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)

        f = io.BytesIO()
        PIL.Image.fromarray(next_frame).save(f, 'jpeg')
        IPython.display.display(IPython.display.Image(data=f.getvalue()))

        in_frame_fd = cv2.resize(next_frame, (w_fd, h_fd))
        #in_frame_fd = in_frame_fd.transpose((2, 0, 1))
        in_frame_fd = in_frame_fd.reshape((n_fd, h_fd, w_fd, c_fd))
        in_frame_fd = preprocess_input(in_frame_fd)

        time3 = time.time()
        # Start inference for specified request
        res = model.predict(in_frame_fd)
        time4 = time.time()

        top = res[0].argsort()[-1:][::-1]
        pred_label = labels[top[0]]

        time2 = time.time()
        fps = '%.1f' % (1/(time2-time1))
        ips = '%.1f' % (1/(time4-time3))

        clear_output(wait=True)
    else:
        break

clear_output(wait=True)
print("Video Ended")
cap.release()
cv2.destroyAllWindows()

## Summary
 
**In this section of the training you learned**
- Create Intermediate Representation (.bin/.xml)
- Understand OpenVINO Arguments
- Instantiate OpenVINO Network
- Use OpenCV to read video and pass frames to OpenVINO